In [11]:
import numpy as np

# =========================
# MODEL PARAMETERS
# =========================
N = 60
S0 = 1000
K = 1040
r = 0.05
T = 1
dt = T / N

u = 1.005   # +0.5%
d = 0.997   # -0.3%

R = 1 + r * dt
p = (R - d) / (u - d)

print("==== MODEL PARAMETERS ====")
print("Number of periods (N):", N)
print("Initial stock price S0:", S0)
print("Strike price K:", K)
print("Interest rate r:", r)
print("dt:", dt)
print("Up factor u:", u)
print("Down factor d:", d)
print("Per-step risk-free growth R:", R)
print("Risk-neutral probability p:", round(p,6))

==== MODEL PARAMETERS ====
Number of periods (N): 60
Initial stock price S0: 1000
Strike price K: 1040
Interest rate r: 0.05
dt: 0.016666666666666666
Up factor u: 1.005
Down factor d: 0.997
Per-step risk-free growth R: 1.0008333333333332
Risk-neutral probability p: 0.479167


In [12]:
def price_european_option(S0, K, r, u, d, N, option_type="call", verbose=False):
    dt = 1 / N
    R = 1 + r * dt
    p = (R - d) / (u - d)

    # Terminal stock prices
    stock_prices = np.array([
        S0 * (u ** j) * (d ** (N - j)) for j in range(N + 1)
    ])

    if verbose:
        print("\nTerminal stock prices (sample):")
        print(stock_prices[:5], "...", stock_prices[-5:])

    # Terminal payoff
    if option_type == "call":
        option_values = np.maximum(stock_prices - K, 0)
    else:
        option_values = np.maximum(K - stock_prices, 0)

    if verbose:
        print("\nTerminal payoffs (sample):")
        print(option_values[:5], "...", option_values[-5:])

    # Backward induction
    for i in range(N, 0, -1):
        option_values = (p * option_values[1:i+1] +
                         (1 - p) * option_values[0:i]) / R

    return option_values[0]

In [13]:
print("==== PART (a) & (b): Prices at t0 ====")

call_price = price_european_option(S0, K, r, u, d, N, "call", verbose=True)
put_price  = price_european_option(S0, K, r, u, d, N, "put", verbose=True)

print("European Call Price at t0 =", round(call_price,4))
print("European Put Price at t0  =", round(put_price,4))

==== PART (a) & (b): Prices at t0 ====

Terminal stock prices (sample):
[835.04426696 841.74472246 848.4989429  855.30735969 862.17040771] ... [1306.41179144 1316.894534   1327.46139084 1338.11303691 1348.85015255]

Terminal payoffs (sample):
[0. 0. 0. 0. 0.] ... [266.41179144 276.894534   287.46139084 298.11303691 308.85015255]

Terminal stock prices (sample):
[835.04426696 841.74472246 848.4989429  855.30735969 862.17040771] ... [1306.41179144 1316.894534   1327.46139084 1338.11303691 1348.85015255]

Terminal payoffs (sample):
[204.95573304 198.25527754 191.5010571  184.69264031 177.82959229] ... [0. 0. 0. 0. 0.]
European Call Price at t0 = 18.3969
European Put Price at t0  = 7.6961


In [14]:
print("\n==== PART (c): Position at t5 ====")

up_moves = 3
down_moves = 2

S_t5 = S0 * (u ** up_moves) * (d ** down_moves)

print("Path: U,U,U,D,D")
print("Stock price at t5 =", round(S_t5,4))


==== PART (c): Position at t5 ====
Path: U,U,U,D,D
Stock price at t5 = 1008.9938


In [15]:
def option_value_and_delta(S, K, r, u, d, N_remaining, option_type):
    dt = 1 / 60
    R = 1 + r * dt
    p = (R - d) / (u - d)

    stock_prices = np.array([
        S * (u ** j) * (d ** (N_remaining - j))
        for j in range(N_remaining + 1)
    ])

    if option_type == "call":
        option_values = np.maximum(stock_prices - K, 0)
    else:
        option_values = np.maximum(K - stock_prices, 0)

    # Backward induction + capture first step for delta
    for i in range(N_remaining, 0, -1):
        if i == 1:
            V_up = option_values[1]
            V_down = option_values[0]
            S_up = S * u
            S_down = S * d
            delta = (V_up - V_down) / (S_up - S_down)

        option_values = (p * option_values[1:i+1] +
                         (1 - p) * option_values[0:i]) / R

    return option_values[0], delta


N_remaining = N - 5

call_t5, delta_call = option_value_and_delta(
    S_t5, K, r, u, d, N_remaining, "call"
)

put_t5, delta_put = option_value_and_delta(
    S_t5, K, r, u, d, N_remaining, "put"
)

print("Call Price at t5 =", round(call_t5,4))
print("Put Price at t5  =", round(put_t5,4))
print("Call Hedge Ratio (Delta) =", round(delta_call,6))
print("Put Hedge Ratio (Delta)  =", round(delta_put,6))

Call Price at t5 = 21.2367
Put Price at t5  = 5.671
Call Hedge Ratio (Delta) = 0.706581
Put Hedge Ratio (Delta)  = -0.293419


In [16]:
print("\n==== PART (d): r = 0 ====")

r_zero = 0

call_price_r0 = price_european_option(S0, K, r_zero, u, d, N, "call")
put_price_r0  = price_european_option(S0, K, r_zero, u, d, N, "put")

call_t5, delta_call = option_value_and_delta(
    S_t5, K, r_zero, u, d, N_remaining, "call"
)

put_t5, delta_put = option_value_and_delta(
    S_t5, K, r_zero, u, d, N_remaining, "put"
)

print("European Call Price at t0 (r=0) =", round(call_price_r0,4))
print("European Put Price at t0  (r=0) =", round(put_price_r0,4))
print("Call price at t5 (r=0) =", round(call_t5,4))
print("Put price at t5  (r=0) =", round(put_t5,4))
print("Call Hedge Ratio at t5 (r=0) =", round(delta_call,4))
print("Put Hedge Ratio at t5  (r=0) =", round(delta_put,4))


==== PART (d): r = 0 ====
European Call Price at t0 (r=0) = 1.4525
European Put Price at t0  (r=0) = 41.4525
Call price at t5 (r=0) = 2.3077
Put price at t5  (r=0) = 33.3138
Call Hedge Ratio at t5 (r=0) = 0.1576
Put Hedge Ratio at t5  (r=0) = -0.8424
